In [1]:
!pip install rouge-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=ac7a0aec5434d01bf85976778e73e8c2c41a44f590539ec45eb2f295c00ba166
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [2]:
!pip install nltk
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [3]:
import pandas as pd
import numpy as np
from rouge_score import rouge_scorer
from nltk.translate import meteor_score
import nltk
from typing import List, Dict, Tuple

In [4]:
class SummaryEvaluator:
    def __init__(self):
        """Initialisieren Sie den Evaluator mit den notwendigen NLTK-Downloads"""
        # Ensure required NLTK data is available
        try:
            nltk.data.find('wordnet')
        except LookupError:
            nltk.download('wordnet')

        # Initialize ROUGE scorer
        self.rouge_scorer = rouge_scorer.RougeScorer(
            ['rouge1', 'rouge2', 'rougeL'],
            use_stemmer=True
        )

    def calculate_rouge_scores(self,
                             candidate_summary: str,
                             reference_summaries: List[str]) -> Dict[str, float]:
        """
        Berechnet ROUGE-Scores für eine Kandidatenzusammenfassung anhand mehrerer Referenzen.

        Args:
            candidate_summary: Die zu bewertende Zusammenfassung
            reference_summaries: Liste der Referenzzusammenfassungen

        Rückgabe:
            Wörterbuch mit ROUGE-1-, ROUGE-2- und ROUGE-L-Bewertungen
        """
        rouge1_scores = []
        rouge2_scores = []
        rougeL_scores = []

        for reference in reference_summaries:
            scores = self.rouge_scorer.score(reference, candidate_summary)

            rouge1_scores.append(scores['rouge1'].fmeasure)
            rouge2_scores.append(scores['rouge2'].fmeasure)
            rougeL_scores.append(scores['rougeL'].fmeasure)

        return {
            'rouge1': max(rouge1_scores),
            'rouge2': max(rouge2_scores),
            'rougeL': max(rougeL_scores)
        }

    def calculate_meteor_score(self,
                             candidate_summary: str,
                             reference_summaries: List[str]) -> float:
        """
        Berechnet den METEOR-Score für eine Kandidatenzusammenfassung anhand mehrerer Referenzen.

        Args:
            candidate_summary: Die zu bewertende Zusammenfassung
            reference_summaries: Liste der Referenzzusammenfassungen

        Rückgabe:
            Maximale METEOR-Punktzahl über alle Referenzen
        """
        candidate_tokens = nltk.word_tokenize(candidate_summary.lower())
        meteor_scores = []

        for reference in reference_summaries:
            reference_tokens = nltk.word_tokenize(reference.lower())
            score = meteor_score.meteor_score([reference_tokens], candidate_tokens)
            meteor_scores.append(score)

        return max(meteor_scores)

    def evaluate_summaries(self,
                         chatgpt_summary: str,
                         system_summary: str,
                         references: List[str]) -> Tuple[pd.DataFrame, Dict]:
        """
        Bewerten Sie beide Zusammenfassungen anhand der ROUGE- und METEOR-Metriken.

        Args:
            chatgpt_summary: Von ChatGPT erzeugte Zusammenfassung
            system_summary: Vom System erzeugte Zusammenfassung
            references: Liste der Referenzzusammenfassungen

        Rückgabe:
            Tupel enthält:
            - DataFrame mit detaillierten ROUGE-Bewertungen
            - Wörterbuch mit METEOR-Bewertungen
        """
        # Calculate ROUGE scores
        chatgpt_rouge = self.calculate_rouge_scores(chatgpt_summary, references)
        system_rouge = self.calculate_rouge_scores(system_summary, references)

        # Calculate METEOR scores
        chatgpt_meteor = self.calculate_meteor_score(chatgpt_summary, references)
        system_meteor = self.calculate_meteor_score(system_summary, references)

        # Create ROUGE comparison DataFrame
        rouge_results = pd.DataFrame({
            'Metric': ['ROUGE-1', 'ROUGE-2', 'ROUGE-L'],
            'ChatGPT': [
                chatgpt_rouge['rouge1'],
                chatgpt_rouge['rouge2'],
                chatgpt_rouge['rougeL']
            ],
            'Your System': [
                system_rouge['rouge1'],
                system_rouge['rouge2'],
                system_rouge['rougeL']
            ]
        })

        # Create METEOR results dictionary
        meteor_results = {
            'ChatGPT METEOR': chatgpt_meteor,
            'System METEOR': system_meteor
        }

        return rouge_results, meteor_results

def save_results(rouge_df: pd.DataFrame,
                meteor_dict: Dict,
                output_file: str = 'evaluation_results.txt'):
    """
    Speichern der Auswertungsergebnisse in einer Datei.

    Args:
        rouge_df: DataFrame mit ROUGE-Bewertungen
        meteor_dict: Wörterbuch mit METEOR-Bewertungen
        ausgabe_datei: Pfad zur Ausgabedatei
    """
    with open(output_file, 'w', encoding='utf-8') as f:
        f.write("=== ROUGE Scores ===\n")
        f.write(rouge_df.to_string())
        f.write("\n\n=== METEOR Scores ===\n")
        for system, score in meteor_dict.items():
            f.write(f"{system}: {score:.4f}\n")

In [5]:
# Testfall "Nur Bild"
if __name__ == "__main__":
    references = [
        "Ein berührender Moment mütterlicher Freude zeigt sich, als eine Frau ihr lächelndes Baby in die Luft hebt.",
        "Eine Mutter hebt freudig ihr lachendes Baby in die Luft.",
        "Das Baby trägt eine weiße Windel und streckt sich spielerisch, während die Mutter es über sich hält und anlächelt"
    ]

    chatgpt_summary = "Eine lächelnde Frau hält ein fröhliches Baby in einer weißen Windel hoch in die Luft."
    system_summary = "eine Frau, die ein Baby in die Luft hält."

    # Initialisiere evaluator
    evaluator = SummaryEvaluator()

    # Berechne scores
    rouge_results, meteor_results = evaluator.evaluate_summaries(
        chatgpt_summary,
        system_summary,
        references
    )

    # Zeig die Ergebnisse
    print("\nROUGE Score Comparison:")
    print(rouge_results.to_string(index=False))
    print("\nMETEOR Score Comparison:")
    for system, score in meteor_results.items():
        print(f"{system}: {score:.4f}")

    # Speichere die Ergebnisse in einer Datei
    save_results(rouge_results, meteor_results, 'evaluation_nur_Bild.txt')

[nltk_data] Downloading package wordnet to /root/nltk_data...



ROUGE Score Comparison:
 Metric  ChatGPT  Your System
ROUGE-1 0.461538     0.500000
ROUGE-2 0.222222     0.333333
ROUGE-L 0.358974     0.500000

METEOR Score Comparison:
ChatGPT METEOR: 0.4891
System METEOR: 0.5114


In [6]:
# Testfall "Nur Text"
if __name__ == "__main__":
    references = [
        "Die Folie beschreibt die zunehmenden Herausforderungen in der Softwaretechnik, wie steigende Komplexität, längere Lebensdauer und wachsende Anforderungen an Anwendungen. Außerdem wird betont, dass die Softwareentwicklung ein zentraler Bestandteil der Systementwicklung ist und die Pflege von Altsystemen immer wichtiger wird.",
        "Es wird erläutert, warum Softwaretechnik wichtig ist, mit Fokus auf die wachsende Komplexität von Programmen und die zunehmende Bedeutung der Softwareentwicklung. Die Pflege von Altsystemen und die längere Lebensdauer von Software stellen weitere Herausforderungen dar.",
        "Die Folie thematisiert die Verschärfung von Problemen in der Softwaretechnik, einschließlich zunehmender Komplexität, längerer Lebenszyklen und wachsender Anforderungen. Entwickler widmen sich zunehmend der Pflege bestehender Altsysteme."
    ]

    chatgpt_summary = "Die Komplexität von Programmen, deren Umfang und Lebensdauer nehmen ständig zu, während neue Anwendungen für den Rechnereinsatz erschlossen werden. Softwareentwicklung ist ein wesentlicher Bestandteil der Systementwicklung, und immer mehr Entwickler sind mit der Pflege von Altsystemen beschäftigt. Dies führt zu einer Verschärfung der Probleme in der Softwareentwicklung."
    system_summary = "Komplexität von Programmen nimmt ständig zu Umfang und Lebensdauer nehmen. Neue Anwendungen werden für den Rechnereinsatz erschlossen. Softwareentwicklung ist ein integraler Bestandteil der Systementwickler."

    evaluator = SummaryEvaluator()

    rouge_results, meteor_results = evaluator.evaluate_summaries(
        chatgpt_summary,
        system_summary,
        references
    )

    print("\nROUGE Score Comparison:")
    print(rouge_results.to_string(index=False))
    print("\nMETEOR Score Comparison:")
    for system, score in meteor_results.items():
        print(f"{system}: {score:.4f}")

    save_results(rouge_results, meteor_results, 'evaluation_nur_Text.txt')


ROUGE Score Comparison:
 Metric  ChatGPT  Your System
ROUGE-1 0.416667     0.318841
ROUGE-2 0.133333     0.095238
ROUGE-L 0.291667     0.231884

METEOR Score Comparison:
ChatGPT METEOR: 0.3287
System METEOR: 0.1744


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [7]:
# Testfall: Bild und Text
if __name__ == "__main__":
    references = [
        "Die Folie erklärt das Wasserfallmodell als ein lineares Entwicklungsmodell, bei dem Produkte wie in einem Wasserfall von einer Phase zur nächsten übergehen. Das Bild eines Wasserfalls verdeutlicht die schrittweise Abfolge, während die Text die Ursprünge und Weiterentwicklungen des Modells erläutert.",
        "Das Wasserfallmodell wird als erstes umfassendes Modell der Softwaretechnik beschrieben, basierend auf dem stagewise model von Benington (1956). Das Bild eines Wasserfalls symbolisiert die lineare Struktur, bei der jede Phase abgeschlossen sein muss, bevor die nächste beginnt.",
        "Die Folie stellt das Wasserfallmodell als ein stufenweises Entwicklungsmodell vor, inspiriert vom natürlichen Ablauf eines Wasserfalls, wie im Bild dargestellt. Ursprünge und Weiterentwicklungen durch Benington, Royce und Boehm werden ebenfalls genannt."
    ]

    chatgpt_summary = "Das Wasserfallmodell beschreibt einen linearen Entwicklungsprozess, bei dem die Ergebnisse einer Phase wie bei einem Wasserfall in die nächste übergehen. Es ist eines der ersten umfassenden Modelle im Software Engineering, basiert auf dem stagewise model von Benington (1956) und wurde durch Royce und Boehm weiterentwickelt. Das Bild eines Wasserfalls verdeutlicht die sequentielle Natur des Prozesses."
    system_summary = "HS Bremen, FK4, Softwaretechnik, ISS & ATMECH, WiSe 23/24, Prof. Dr. -Ing. J. Matevska 11 Wasserfallmodell. Basiert auf dem stagewise model von Benington [1956] Weiterentwicklung u. a. durch Royce [1970 & 1987] und Boehm [1981].Das Bild zeigt ein kleiner Wasserfall mitten in einem Wald."

    evaluator = SummaryEvaluator()

    rouge_results, meteor_results = evaluator.evaluate_summaries(
        chatgpt_summary,
        system_summary,
        references
    )

    print("\nROUGE Score Comparison:")
    print(rouge_results.to_string(index=False))
    print("\nMETEOR Score Comparison:")
    for system, score in meteor_results.items():
        print(f"{system}: {score:.4f}")

    save_results(rouge_results, meteor_results, 'evaluation_Bild_und_Text.txt')


ROUGE Score Comparison:
 Metric  ChatGPT  Your System
ROUGE-1 0.495050     0.317073
ROUGE-2 0.260870     0.175000
ROUGE-L 0.319149     0.292683

METEOR Score Comparison:
ChatGPT METEOR: 0.5322
System METEOR: 0.3079


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [8]:
# Testfall: komplexe Bilder
if __name__ == "__main__":
    references = [
        "Während Änderungen in der Anforderungsdefinition kostengünstig sind (1x), steigen sie in der Entwicklung (1,5-6x) und explodieren in der Wartung (60-100x).",
        "Eine gründliche Anforderungsanalyse minimiert Kostensteigerungen. Änderungen sind in der frühen Phase kostengünstig (1x), aber während der Wartung können sie bis zu 100-mal teurer werden",
        "Anforderungen frühzeitig zu definieren, ist essenziell, da die Kosten für Änderungen im Entwicklungsprozess exponentiell steigen. Während sie in der Anforderungsdefinition minimal sind, steigen sie in der Entwicklung um das 1,5- bis 6-fache und in der Wartung um das 60- bis 100-fache"
    ]

    chatgpt_summary = "Die Folie zeigt, warum Anforderungs-Engineering wichtig ist, anhand der steigenden Kosten von Änderungen im Entwicklungsprozess. Änderungen während der Anforderungsdefinition kosten am wenigsten (1x), während sie in der Entwicklung (1,5–6x) und besonders in der Wartung (60–100x) exponentiell teurer werden."
    system_summary = "HS Bremen, FK4, TI, Softwaretechnik 2, SoSe 2023, Prof. -Ing. J. Matevska 16 Warum ist Anforderungs-Engineering notwendig.Das Bild zeigt ein Balkendiagramm, das den Prozentsatz der Anzahl der diagnostizierten Personen zeigt."

    evaluator = SummaryEvaluator()

    rouge_results, meteor_results = evaluator.evaluate_summaries(
        chatgpt_summary,
        system_summary,
        references
    )

    print("\nROUGE Score Comparison:")
    print(rouge_results.to_string(index=False))
    print("\nMETEOR Score Comparison:")
    for system, score in meteor_results.items():
        print(f"{system}: {score:.4f}")

    save_results(rouge_results, meteor_results, 'evaluation_komplexe_Bilder.txt')


ROUGE Score Comparison:
 Metric  ChatGPT  Your System
ROUGE-1 0.550725     0.126582
ROUGE-2 0.388060     0.000000
ROUGE-L 0.521739     0.096774

METEOR Score Comparison:
ChatGPT METEOR: 0.6702
System METEOR: 0.0903


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [9]:
# Testfall: Folie mit mathematischen Formeln
if __name__ == "__main__":
    references = ["Der magnetische Fluss folgt einer sinusförmigen Funktion, die durch Φ=Φmaxcos(𝜔𝑡) beschrieben wird. Laut Induktionsgesetz induziert eine rotierende Spule in einem Magnetfeld eine sinusförmige Wechselspannung 𝑢(𝑡)=ûsin(𝜔𝑡), wobei û=𝑁𝜔Φmax. Die Frequenz f ist der Kehrwert der Periodendauer T, und die Kreisfrequenz ω ist 2𝜋𝑓.",
        "Eine Spule, die in einem homogenen Magnetfeld gedreht wird, erzeugt eine sinusförmige Wechselspannung. Der Zusammenhang zwischen Frequenz (f), Periodendauer (T) und Kreisfrequenz (ω) wird durch 𝜔=2𝜋𝑓 und 𝑇=1/𝑓 beschrieben. Der magnetische Fluss verändert sich sinusförmig mit der Zeit.",
        "Der magnetische Fluss Φ und die induzierte Spannung 𝑢(𝑡)sind zeitabhängig und folgen sinusförmigen Gesetzmäßigkeiten. Die Kreisfrequenz ω und die Frequenz f beschreiben die zeitliche Veränderung der Spannung, wobei 𝑇=1/f die Periodendauer angibt. Die Induktion basiert auf der Rotation der Spule im Magnetfeld."
    ]
    chatgpt_summary = "Die Folie erklärt, dass der magnetische Fluss in einer Spule eine sinusförmige Spannung induziert, wenn die Spule in einem homogenen Magnetfeld rotiert. Die Spannung hängt von der Windungszahl N, der Winkelgeschwindigkeit ω, und der maximalen Flussdichte Φmax ab, mit einer Periodendauer T, die in Sekunden gemessen wird."
    system_summary = "Für den magnetischen Fluss ergibt sich somit Φmax cosω. Die Zeit für einen vollen Umlauf wird Periodendauer genannt. Die Einheit ist die Sekunde."

    evaluator = SummaryEvaluator()

    rouge_results, meteor_results = evaluator.evaluate_summaries(
        chatgpt_summary,
        system_summary,
        references
    )

    print("\nROUGE Score Comparison:")
    print(rouge_results.to_string(index=False))
    print("\nMETEOR Score Comparison:")
    for system, score in meteor_results.items():
        print(f"{system}: {score:.4f}")

    save_results(rouge_results, meteor_results, 'evaluation_Folie_mit_mathFormeln.txt')


ROUGE Score Comparison:
 Metric  ChatGPT  Your System
ROUGE-1 0.478261     0.264706
ROUGE-2 0.206897     0.000000
ROUGE-L 0.347826     0.205882

METEOR Score Comparison:
ChatGPT METEOR: 0.3573
System METEOR: 0.1160


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
